In [13]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from numba import jit, prange
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')

# GPU-accelerated libraries (fallback to CPU if not available)
try:
    import cudf as df_lib
    import cupy as np_gpu
    USE_GPU = True
    print("Using GPU acceleration with cuDF/cuPy")
except ImportError:
    import pandas as df_lib
    import numpy as np_gpu
    USE_GPU = False
    print("Using CPU with pandas/numpy")

Using CPU with pandas/numpy


In [14]:
# Imposta il percorso dei dati (modificabile globalmente)
DATA_PATH = "/workspaces/bollingerBands/notebooks/DATA/"

# Se vuoi cambiare la cartella dei dati, modifica la variabile DATA_PATH sopra.

In [15]:
@jit(nopython=True)
def normalize_scores(returns, method='minmax'):
    """Normalizza i rendimenti tra 0 e 1"""
    if len(returns) == 0:
        return returns
    min_val = np.min(returns)
    max_val = np.max(returns)
    if max_val == min_val:
        return np.ones_like(returns) / len(returns)
    return (returns - min_val) / (max_val - min_val)

@jit(nopython=True)
def calculate_momentum_weights(returns_matrix, lookback):
    """
    Calcola i pesi basati su momentum normalizzato.
    Esclude completamente le strategie in perdita (rendimento negativo).
    Se tutte sono in perdita, restituisce un array di zeri (non investire).
    """
    n_assets = returns_matrix.shape[1]
    weights = np.zeros(n_assets)
    
    if len(returns_matrix) < lookback:
        return np.ones(n_assets) / n_assets
    
    # Calcola rendimenti cumulati nell'ultimo lookback
    recent_returns = returns_matrix[-lookback:]
    cum_returns = np.zeros(n_assets)
    
    for i in range(n_assets):
        cum_returns[i] = np.prod(1 + recent_returns[:, i]) - 1
    
    # Esclude strategie in perdita (rendimento cumulativo <= 0)
    positive_returns_mask = cum_returns > 0
    
    # Se tutte le strategie sono in perdita, non investire in nessuna
    if not np.any(positive_returns_mask):
        return np.zeros(n_assets)
    
    # Considera solo strategie in profitto
    filtered_returns = np.where(positive_returns_mask, cum_returns, 0)
    
    # Normalizza tra 0 e 1
    weights = normalize_scores(filtered_returns)
    
    # Assicura che non ci siano pesi per strategie in perdita
    weights = np.where(positive_returns_mask, weights, 0)
    
    # Assicura che la somma sia 1
    total = np.sum(weights)
    if total > 0:
        weights = weights / total
    
    return weights

@jit(nopython=True)
def calculate_sharpe_momentum_weights(returns_matrix, lookback):
    """
    Calcola i pesi basati su Sharpe-adjusted momentum.
    Usa lo Sharpe ratio per pesare le strategie invece dei semplici rendimenti.
    Esclude completamente le strategie con Sharpe negativo.
    Se tutte hanno Sharpe negativo, restituisce un array di zeri (non investire).
    """
    n_assets = returns_matrix.shape[1]
    weights = np.zeros(n_assets)
    
    if len(returns_matrix) < lookback:
        return np.ones(n_assets) / n_assets
    
    # Calcola Sharpe ratio per ogni strategia nell'ultimo lookback
    recent_returns = returns_matrix[-lookback:]
    sharpe_ratios = np.zeros(n_assets)
    
    for i in range(n_assets):
        asset_returns = recent_returns[:, i]
        
        # Calcola media e std dei rendimenti
        mean_return = np.mean(asset_returns)
        std_return = np.std(asset_returns)
        
        # Calcola Sharpe ratio (assumendo risk-free rate = 0)
        if std_return > 0:
            # Annualizza il Sharpe ratio (assumendo dati giornalieri)
            sharpe_ratios[i] = (mean_return * 252) / (std_return * np.sqrt(252))
        else:
            sharpe_ratios[i] = 0.0
    
    # Esclude strategie con Sharpe negativo o zero
    positive_sharpe_mask = sharpe_ratios > 0
    
    # Se tutte le strategie hanno Sharpe negativo, non investire in nessuna
    if not np.any(positive_sharpe_mask):
        return np.zeros(n_assets)
    
    # Considera solo strategie con Sharpe positivo
    filtered_sharpe = np.where(positive_sharpe_mask, sharpe_ratios, 0)
    
    # Normalizza tra 0 e 1
    weights = normalize_scores(filtered_sharpe)
    
    # Assicura che non ci siano pesi per strategie con Sharpe negativo
    weights = np.where(positive_sharpe_mask, weights, 0)
    
    # Assicura che la somma sia 1
    total = np.sum(weights)
    if total > 0:
        weights = weights / total
    
    return weights

import os

def load_trading_data():
    """Carica tutti i file CSV di trading dalla cartella DATA_PATH"""
    data_path = DATA_PATH
    
    # Trova tutti i file CSV nella cartella
    files = [f for f in os.listdir(data_path) if f.lower().endswith('.csv')]
    if not files:
        print(f"Nessun file CSV trovato in {data_path}")
        return {}, None, None
    
    # Dizionari per tenere traccia dei dati separati
    dfs = {}
    strategy_names = []
    
    for file in files:
        try:
            # Leggi file direttamente senza pd.read_csv per gestire meglio l'encoding UTF-16
            with open(os.path.join(data_path, file), 'r', encoding='utf-16') as f:
                lines = f.readlines()
            
            # Estrai dati dal file
            dates = []
            balances = []
            
            for line in lines[1:]:  # Salta l'header
                parts = line.strip().split('\t')
                if len(parts) >= 2:  # Verifica che ci siano almeno 2 parti
                    date_str = parts[0].strip()
                    try:
                        balance = float(parts[1].strip())
                        dates.append(date_str)
                        balances.append(balance)
                    except ValueError:
                        continue
            
            # Crea DataFrame e assicurati che l'indice sia unico
            df = pd.DataFrame({
                'BALANCE': balances
            }, index=pd.to_datetime(dates))
            
            # Ordina l'indice e gestisci i duplicati (prendi l'ultimo valore per ogni data)
            df = df.sort_index()
            df = df[~df.index.duplicated(keep='last')]
            
            strategy_name = file.split('_')[0].upper()
            dfs[strategy_name] = df
            strategy_names.append(strategy_name)
            
            print(f"Caricato {file}: {len(df)} righe")
            
        except Exception as e:
            print(f"Errore caricando {file}: {e}")
    
    # Combinare tutti i dataframe con un outer join
    combined_df = None
    for name, df in dfs.items():
        if combined_df is None:
            combined_df = df.rename(columns={'BALANCE': name})
        else:
            combined_df = combined_df.join(df.rename(columns={'BALANCE': name}), how='outer')
    
    # Forward fill per i valori mancanti
    combined_df = combined_df.fillna(method='ffill')
    
    # Backward fill per i valori iniziali mancanti
    combined_df = combined_df.fillna(method='bfill')
    
    # Resample a frequenza giornaliera per evitare buchi e assicurare dati regolari
    combined_df = combined_df.resample('D').last().fillna(method='ffill')
    
    # Calcola i rendimenti
    returns_df = combined_df.pct_change().fillna(0)
    
    # Filtra rendimenti outlier
    for col in returns_df.columns:
        returns_df[col] = np.where(
            (returns_df[col] < -0.5) | (returns_df[col] > 0.5),
            0,  # Sostituisci outlier con 0
            returns_df[col]
        )
    
    # Crea dizionario finale delle strategie
    strategies = {}
    for name in strategy_names:
        strategies[name] = pd.DataFrame({
            'BALANCE': combined_df[name],
            'returns': returns_df[name]
        })
    
    print(f"\nDataFrame combinato: {combined_df.shape[0]} righe, {combined_df.shape[1]} colonne")
    
    # Verifica l'unicità dell'indice
    print(f"L'indice è unico: {returns_df.index.is_unique}")
    
    return strategies, combined_df, returns_df

# Carica i dati
print("Caricamento dati...")
strategies_data, combined_df, returns_df = load_trading_data()
print(f"Caricate {len(strategies_data)} strategie: {list(strategies_data.keys())}")

Caricamento dati...
Caricato eurchf_1440_01.csv: 14360 righe
Caricato eurgbp_1440_01.csv: 14096 righe
Caricato nzdjpy_7200_02.csv: 10785 righe
Caricato gbpchf_1440_01.csv: 13567 righe
Caricato eurusd_1440_01.csv: 12346 righe
Caricato audjpy_1440_01.csv: 11820 righe
Caricato audnzd_1440_01.csv: 14460 righe
Caricato gbpusd_1440_01.csv: 12282 righe
Caricato usdcad_1440_01.csv: 12649 righe
Caricato usdchf_1440_01.csv: 12918 righe

DataFrame combinato: 2380 righe, 10 colonne
L'indice è unico: True
Caricate 10 strategie: ['EURCHF', 'EURGBP', 'NZDJPY', 'GBPCHF', 'EURUSD', 'AUDJPY', 'AUDNZD', 'GBPUSD', 'USDCAD', 'USDCHF']


In [16]:
class DynamicPortfolioRebalancer:
    def __init__(self, returns_df, rebalance_frequency=7):
        """
        Inizializza il rebalancer usando il DataFrame dei rendimenti combinato
        
        Args:
            returns_df: DataFrame con i rendimenti di ogni strategia
            rebalance_frequency: frequenza di ribilanciamento in giorni (es. 7 per settimanale)
        """
        self.returns_df = returns_df
        self.rebalance_frequency = rebalance_frequency  # giorni
        self.strategy_names = list(returns_df.columns)
        
        # Verifica che l'indice sia unico
        if not returns_df.index.is_unique:
            print("ATTENZIONE: Indice non unico, potrebbe causare problemi!")
            self.returns_df = self.returns_df[~self.returns_df.index.duplicated(keep='last')]
        
        # Converti il DataFrame in una matrice numpy
        self.returns_matrix = self.returns_df.values  # [giorni, strategie]
        self.dates = self.returns_df.index
        
        print(f"Matrice rendimenti: {self.returns_matrix.shape} - da {self.dates.min()} a {self.dates.max()}")
    
    def _calculate_portfolio_performance(self, returns_matrix, weights_history, rebalance_dates_idx):
        """Calcola performance del portfolio"""
        n_days = returns_matrix.shape[0]
        portfolio_value = np.ones(n_days + 1)  # Includi valore iniziale = 1.0
        portfolio_returns = np.zeros(n_days)
        
        current_weights = weights_history[0] if len(weights_history) > 0 else np.ones(returns_matrix.shape[1]) / returns_matrix.shape[1]
        rebal_idx = 0
        
        for day in range(n_days):
            # Ribilancia se necessario
            if day in rebalance_dates_idx and rebal_idx < len(weights_history):
                current_weights = weights_history[rebal_idx]
                rebal_idx += 1
            
            # Calcola rendimento giornaliero del portfolio
            daily_return = np.dot(current_weights, returns_matrix[day])
            portfolio_returns[day] = daily_return
            portfolio_value[day + 1] = portfolio_value[day] * (1 + daily_return)
        
        return portfolio_returns, portfolio_value
    
    def backtest_strategy(self, lookback_days, method='momentum'):
        """Backtest della strategia di ribilanciamento"""
        n_days = len(self.returns_matrix)
        
        # Verifica che ci siano abbastanza dati
        if n_days < lookback_days:
            print(f"AVVISO: Non ci sono abbastanza dati ({n_days} giorni) per lookback di {lookback_days}!")
            lookback_days = max(5, n_days // 10)  # Usa almeno 5 giorni di lookback
            print(f"Utilizzando lookback ridotto: {lookback_days}")
        
        weights_history = []
        rebalance_dates_idx = []
        rebalance_dates = []
        
        # Trova date di ribilanciamento a intervalli regolari (es. ogni domenica)
        # Usa date fisse per garantire che siano le stesse indipendentemente dai dati
        
        # Genera una sequenza di indici per il ribilanciamento
        # (dopo il periodo di lookback)
        for i in range(lookback_days, n_days, self.rebalance_frequency):
            rebalance_dates_idx.append(i)
            rebalance_dates.append(self.dates[i])
            
            if method == 'momentum':
                # Usa rendimenti fino a questa data
                weights = calculate_momentum_weights(
                    self.returns_matrix[:i], lookback_days
                )
            elif method == 'sharpe_momentum':
                # Nuovo metodo basato su Sharpe ratio
                weights = calculate_sharpe_momentum_weights(
                    self.returns_matrix[:i], lookback_days
                )
            elif method == 'equal':
                weights = self._calculate_equal_weights_exclude_losing(i, lookback_days)
            elif method == 'risk_parity':
                weights = self._calculate_risk_parity_weights(i, lookback_days)
            
            weights_history.append(weights)
        
        # Verifica che ci siano pesi calcolati
        if not weights_history:
            print("Nessun periodo di ribilanciamento trovato!")
            weights_history = [np.ones(len(self.strategy_names)) / len(self.strategy_names)]
            
            if len(rebalance_dates_idx) == 0:
                rebalance_dates_idx = [lookback_days]
                rebalance_dates = [self.dates[lookback_days]]
        
        # Calcola performance
        weights_history = np.array(weights_history)
        
        portfolio_returns, portfolio_values = self._calculate_portfolio_performance(
            self.returns_matrix, weights_history, rebalance_dates_idx
        )
        
        # Crea DataFrame con i risultati
        portfolio_data = pd.DataFrame({
            'returns': portfolio_returns,
            'value': portfolio_values[1:]  # Elimina il valore iniziale
        }, index=self.dates)
        
        # Crea DataFrame con i pesi
        weights_df = pd.DataFrame(
            weights_history, 
            columns=self.strategy_names,
            index=rebalance_dates
        )
        
        return {
            'portfolio_data': portfolio_data,
            'weights': weights_df,
            'final_value': portfolio_values[-1],
            'lookback': lookback_days,
            'method': method
        }
    
    def _calculate_equal_weights_exclude_losing(self, current_day, lookback):
        """
        Calcola pesi equi escludendo le strategie in perdita
        """
        if current_day < lookback:
            return np.ones(len(self.strategy_names)) / len(self.strategy_names)
        
        # Calcola rendimenti cumulati nell'ultimo lookback
        recent_returns = self.returns_matrix[current_day-lookback:current_day]
        cum_returns = np.zeros(len(self.strategy_names))
        
        for i in range(len(self.strategy_names)):
            cum_returns[i] = np.prod(1 + recent_returns[:, i]) - 1
        
        # Identifica strategie in profitto (cum_return > 0)
        profitable = cum_returns > 0
        n_profitable = np.sum(profitable)
        
        # Se tutte sono in perdita, non investire in nessuna
        if n_profitable == 0:
            return np.zeros(len(self.strategy_names))
        
        # Calcola pesi uguali per le strategie redditizie
        weights = np.zeros(len(self.strategy_names))
        weights[profitable] = 1.0 / n_profitable
        
        return weights
    
    def _calculate_risk_parity_weights(self, current_day, lookback):
        """
        Calcola pesi basati su risk parity, escludendo strategie in perdita
        """
        if current_day < lookback:
            return np.ones(len(self.strategy_names)) / len(self.strategy_names)
        
        # Calcola rendimenti cumulati e volatilità nell'ultimo lookback
        recent_returns = self.returns_matrix[current_day-lookback:current_day]
        cum_returns = np.zeros(len(self.strategy_names))
        volatilities = np.std(recent_returns, axis=0)
        
        for i in range(len(self.strategy_names)):
            cum_returns[i] = np.prod(1 + recent_returns[:, i]) - 1
        
        # Identifica strategie in profitto (cum_return > 0)
        profitable = cum_returns > 0
        
        # Se tutte sono in perdita, non investire in nessuna
        if not np.any(profitable):
            return np.zeros(len(self.strategy_names))
        
        # Evita divisione per zero e considera solo strategie profittevoli
        masked_volatilities = np.where(profitable & (volatilities > 0), volatilities, np.inf)
        
        # Pesi inversamente proporzionali alla volatilità
        inv_vol = np.where(masked_volatilities < np.inf, 1.0 / masked_volatilities, 0.0)
        total_inv_vol = np.sum(inv_vol)
        
        # Normalizza i pesi
        weights = np.zeros(len(self.strategy_names))
        if total_inv_vol > 0:
            weights = inv_vol / total_inv_vol
        
        return weights

# Inizializza il rebalancer usando il DataFrame dei rendimenti
rebalancer = DynamicPortfolioRebalancer(returns_df)

Matrice rendimenti: (2380, 10) - da 2019-01-01 00:00:00 a 2025-07-07 00:00:00


In [17]:
def calculate_performance_metrics(returns_series):
    """Calcola metriche di performance"""
    returns = returns_series.values
    if len(returns) < 5:
        return 0.0, 0.0, 0.0, 0.0, 0.0
    
    # Rendimento totale
    total_return = np.prod(1 + returns) - 1
    
    # Rendimento annualizzato (assumendo dati giornalieri)
    n_years = len(returns) / 252
    annual_return = (1 + total_return) ** (1/n_years) - 1 if n_years > 0 else 0
    
    # Volatilità annualizzata
    volatility = np.std(returns) * np.sqrt(252)
    
    # Sharpe ratio
    sharpe = annual_return / volatility if volatility > 0 else 0
    
    # Max drawdown
    cumulative = np.cumprod(1 + returns)
    peak = np.maximum.accumulate(cumulative)
    drawdown = (cumulative - peak) / peak
    max_dd = np.min(drawdown) if len(drawdown) > 0 else 0
    
    return total_return, annual_return, volatility, sharpe, max_dd

def optimize_single_config(lookback, method, rebalancer):
    """Funzione per ottimizzazione parallela"""
    try:
        result = rebalancer.backtest_strategy(lookback, method)
        portfolio_data = result['portfolio_data']
        
        # Calcola metriche
        total_ret, annual_ret, vol, sharpe, max_dd = calculate_performance_metrics(portfolio_data['returns'])
        
        return {
            'lookback': lookback,
            'method': method,
            'total_return': total_ret,
            'annual_return': annual_ret,
            'volatility': vol,
            'sharpe_ratio': sharpe,
            'max_drawdown': max_dd,
            'final_value': result['final_value'],
            'result': result
        }
    except Exception as e:
        print(f"Errore con lookback={lookback}, method={method}: {e}")
        return None

def grid_search_optimization(rebalancer, n_jobs=1):
    """Grid search parallelo per trovare i migliori parametri"""
    
    # Parametri da testare (incluso il nuovo sharpe_momentum)
    lookback_range = [5, 10, 15, 20, 30, 45, 60, 90, 120, 180]
    methods = ['momentum', 'sharpe_momentum', 'equal', 'risk_parity']
    
    # Crea tutte le combinazioni
    configs = []
    for lookback in lookback_range:
        for method in methods:
            configs.append((lookback, method, rebalancer))
    
    print(f"Testing {len(configs)} configurations...")
    
    # Esegui ottimizzazione parallela
    if n_jobs > 1:
        results = Parallel(n_jobs=n_jobs, verbose=1)(
            delayed(optimize_single_config)(*config) for config in configs
        )
    else:
        results = []
        for config in configs:
            result = optimize_single_config(*config)
            if result:
                results.append(result)
    
    # Filtra risultati validi
    valid_results = [r for r in results if r is not None]
    
    if not valid_results:
        print("Nessun risultato valido trovato!")
        return pd.DataFrame()
    
    # Converti in DataFrame per analisi
    results_df = pd.DataFrame(valid_results)
    
    return results_df

# Esegui grid search
print("Inizio ottimizzazione...")
results_df = grid_search_optimization(rebalancer)

if len(results_df) > 0:
    print(f"\nCompletate {len(results_df)} configurazioni")
    print("\nTop 5 per Sharpe Ratio:")
    top_sharpe = results_df.nlargest(5, 'sharpe_ratio')[['lookback', 'method', 'sharpe_ratio', 'annual_return', 'max_drawdown']]
    print(top_sharpe.to_string(index=False))
else:
    print("Nessun risultato valido trovato!")

Inizio ottimizzazione...
Testing 40 configurations...

Completate 40 configurazioni

Top 5 per Sharpe Ratio:
 lookback          method  sharpe_ratio  annual_return  max_drawdown
      180 sharpe_momentum      0.641103       0.002070     -0.006263
      180     risk_parity      0.621678       0.001744     -0.004200
      120     risk_parity      0.457693       0.001324     -0.007448
      180           equal      0.448884       0.001391     -0.005542
      180        momentum      0.442315       0.001556     -0.006850


# Regola di Esclusione delle Strategie in Perdita

Il sistema di ribilanciamento dinamico è stato configurato con una regola fondamentale:

1. **Esclusione delle strategie in perdita**: Non investiamo mai in strategie che hanno un rendimento negativo durante il periodo di lookback
2. **Gestione caso limite**: Se tutte le strategie sono in perdita, manteniamo il capitale in liquidità (pesi = 0)

Questo approccio è implementato in tutti i metodi di allocazione (momentum, equal weight, risk parity) e fornisce una protezione aggiuntiva contro i drawdown.

# Strategie di Allocazione Implementate

Il sistema di ribilanciamento dinamico include diverse strategie di allocazione:

## 1. **Momentum Tradizionale**
- Alloca basandosi sui rendimenti cumulati nel periodo di lookback
- Esclude strategie con rendimento negativo

## 2. **Sharpe-Adjusted Momentum** ⭐ **NUOVO**
- Alloca basandosi sullo **Sharpe ratio** nel periodo di lookback
- Considera sia rendimento che rischio (volatilità)
- Formula: `Sharpe = (Rendimento Medio Annualizzato) / (Volatilità Annualizzata)`
- Esclude strategie con Sharpe ratio negativo o zero
- **Vantaggio**: Preferisce strategie con rendimenti più stabili e consistenti

## 3. **Equal Weight**
- Peso uguale per tutte le strategie in profitto

## 4. **Risk Parity**
- Peso inversamente proporzionale alla volatilità

## Regola Comune di Esclusione
- **Tutte le strategie** escludono automaticamente asset in perdita/sottoperformanti
- Se tutti gli asset sono in perdita, il capitale rimane in liquidità (pesi = 0)

In [18]:
# Visualizza i risultati della migliore strategia
if len(results_df) > 0:
    # Ottieni la migliore strategia per Sharpe ratio
    best_idx = results_df['sharpe_ratio'].idxmax()
    best_strategy = results_df.loc[best_idx]
    
    # Recupera i risultati completi
    result = best_strategy['result']
    portfolio_data = result['portfolio_data']
    weights_df = result['weights']
    
    print(f"\nMiglior strategia: {best_strategy['method']} con lookback={best_strategy['lookback']}")
    print(f"Rendimento annuale: {best_strategy['annual_return']:.2%}")
    print(f"Sharpe Ratio: {best_strategy['sharpe_ratio']:.2f}")
    print(f"Max Drawdown: {best_strategy['max_drawdown']:.2%}")
    
    # Grafico della performance
    fig = make_subplots(
        rows=2, cols=1, 
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=("Performance del Portfolio", "Allocazione dei Pesi")
    )
    
    # Performance chart
    fig.add_trace(
        go.Scatter(
            x=portfolio_data.index, 
            y=portfolio_data['value'],
            mode='lines',
            name='Portfolio Value'
        ),
        row=1, col=1
    )
    
    # Heatmap per i pesi
    weights_df_resampled = weights_df.resample('M').ffill()  # Ricampiona per leggibilità
    
    # Plot dei pesi nel tempo come aree impilate
    for col in weights_df_resampled.columns:
        fig.add_trace(
            go.Scatter(
                x=weights_df_resampled.index,
                y=weights_df_resampled[col],
                mode='lines',
                stackgroup='one',
                name=col
            ),
            row=2, col=1
        )
    
    fig.update_layout(
        height=800,
        title=f"Strategia di Ribilanciamento Dinamico ({best_strategy['method']} - lookback={best_strategy['lookback']} giorni)",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.3,
            xanchor="center",
            x=0.5
        )
    )
    
    fig.show()
    
    # Tabella dei pesi finali
    print("\nPesi finali:")
    last_weights = weights_df.iloc[-1].sort_values(ascending=False)
    for asset, weight in last_weights.items():
        print(f"{asset}: {weight:.2%}")
        
    # Confronta con altre strategie
    best_methods = {}
    for method in results_df['method'].unique():
        best_method_idx = results_df[results_df['method'] == method]['sharpe_ratio'].idxmax()
        best_methods[method] = results_df.loc[best_method_idx]
    
    # Crea un grafico per confrontare le strategie
    fig2 = go.Figure()
    
    for method, data in best_methods.items():
        result = data['result']
        portfolio_data = result['portfolio_data']
        fig2.add_trace(
            go.Scatter(
                x=portfolio_data.index,
                y=portfolio_data['value'],
                mode='lines',
                name=f"{method} (lookback={data['lookback']})"
            )
        )
    
    fig2.update_layout(
        title="Confronto tra Strategie di Ribilanciamento",
        xaxis_title="Data",
        yaxis_title="Valore Portfolio",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        )
    )
    
    fig2.show()


Miglior strategia: sharpe_momentum con lookback=180
Rendimento annuale: 0.21%
Sharpe Ratio: 0.64
Max Drawdown: -0.63%



Pesi finali:
EURCHF: 36.43%
AUDNZD: 32.39%
USDCAD: 12.07%
EURUSD: 7.97%
NZDJPY: 5.20%
EURGBP: 4.02%
GBPUSD: 1.34%
GBPCHF: 0.58%
AUDJPY: 0.00%
USDCHF: 0.00%


In [19]:
# Analisi di sensibilità ai parametri per la migliore strategia
if len(results_df) > 0:
    # Ottieni il metodo della migliore strategia
    best_method = best_strategy['method']
    
    print(f"\n{'='*60}")
    print(f"ANALISI SENSIBILITÀ PARAMETRI - METODO: {best_method.upper()}")
    print(f"{'='*60}")
    
    # Filtra solo le strategie con il metodo migliore
    method_results = results_df[results_df['method'] == best_method].copy()
    method_results = method_results.sort_values('lookback')
    
    # Crea tabella di confronto
    sensitivity_table = method_results[['lookback', 'sharpe_ratio', 'annual_return', 'volatility', 'max_drawdown']].copy()
    sensitivity_table = sensitivity_table.round(4)
    
    print(f"\nPerformance per diversi lookback (metodo {best_method}):")
    print("-" * 70)
    for _, row in sensitivity_table.iterrows():
        lookback = int(row['lookback'])
        sharpe = row['sharpe_ratio']
        annual_ret = row['annual_return']
        vol = row['volatility'] 
        max_dd = row['max_drawdown']
        
        # Evidenzia la migliore configurazione
        marker = " ⭐ BEST" if lookback == best_strategy['lookback'] else ""
        print(f"Lookback {lookback:3d}: Sharpe={sharpe:6.3f} | Annual={annual_ret:7.2%} | Vol={vol:6.2%} | MaxDD={max_dd:7.2%}{marker}")
    
    # Calcola statistiche di sensibilità
    sharpe_std = method_results['sharpe_ratio'].std()
    sharpe_mean = method_results['sharpe_ratio'].mean()
    cv_sharpe = sharpe_std / sharpe_mean if sharpe_mean != 0 else 0
    
    annual_ret_std = method_results['annual_return'].std()
    annual_ret_mean = method_results['annual_return'].mean()
    cv_annual = annual_ret_std / annual_ret_mean if annual_ret_mean != 0 else 0
    
    print(f"\n📊 STATISTICHE DI SENSIBILITÀ:")
    print(f"   Sharpe Ratio - Media: {sharpe_mean:.3f}, Std: {sharpe_std:.3f}, CV: {cv_sharpe:.2%}")
    print(f"   Annual Return - Media: {annual_ret_mean:.2%}, Std: {annual_ret_std:.3f}, CV: {cv_annual:.2%}")
    
    # Interpretazione della sensibilità
    if cv_sharpe < 0.2:
        sensitivity_level = "BASSA"
        interpretation = "La strategia è robusta ai cambi di parametri"
    elif cv_sharpe < 0.5:
        sensitivity_level = "MEDIA"
        interpretation = "La strategia mostra moderata sensibilità ai parametri"
    else:
        sensitivity_level = "ALTA"
        interpretation = "La strategia è molto sensibile ai parametri"
    
    print(f"\n🎯 SENSIBILITÀ AI PARAMETRI: {sensitivity_level}")
    print(f"   {interpretation}")
    
    # Grafico della sensibilità
    fig_sensitivity = make_subplots(
        rows=2, cols=2,
        subplot_titles=("Sharpe Ratio vs Lookback", "Annual Return vs Lookback", 
                       "Volatility vs Lookback", "Max Drawdown vs Lookback"),
        vertical_spacing=0.1,
        horizontal_spacing=0.1
    )
    
    # Sharpe Ratio
    fig_sensitivity.add_trace(
        go.Scatter(
            x=method_results['lookback'],
            y=method_results['sharpe_ratio'],
            mode='lines+markers',
            name='Sharpe Ratio',
            line=dict(color='blue', width=2),
            marker=dict(size=8)
        ),
        row=1, col=1
    )
    
    # Evidenzia il best
    best_row = method_results[method_results['lookback'] == best_strategy['lookback']].iloc[0]
    fig_sensitivity.add_trace(
        go.Scatter(
            x=[best_row['lookback']],
            y=[best_row['sharpe_ratio']],
            mode='markers',
            name='Best Config',
            marker=dict(size=15, color='red', symbol='star')
        ),
        row=1, col=1
    )
    
    # Annual Return
    fig_sensitivity.add_trace(
        go.Scatter(
            x=method_results['lookback'],
            y=method_results['annual_return'] * 100,  # Converti in percentuale
            mode='lines+markers',
            name='Annual Return (%)',
            line=dict(color='green', width=2),
            marker=dict(size=8),
            showlegend=False
        ),
        row=1, col=2
    )
    
    fig_sensitivity.add_trace(
        go.Scatter(
            x=[best_row['lookback']],
            y=[best_row['annual_return'] * 100],
            mode='markers',
            marker=dict(size=15, color='red', symbol='star'),
            showlegend=False
        ),
        row=1, col=2
    )
    
    # Volatility
    fig_sensitivity.add_trace(
        go.Scatter(
            x=method_results['lookback'],
            y=method_results['volatility'] * 100,
            mode='lines+markers',
            name='Volatility (%)',
            line=dict(color='orange', width=2),
            marker=dict(size=8),
            showlegend=False
        ),
        row=2, col=1
    )
    
    fig_sensitivity.add_trace(
        go.Scatter(
            x=[best_row['lookback']],
            y=[best_row['volatility'] * 100],
            mode='markers',
            marker=dict(size=15, color='red', symbol='star'),
            showlegend=False
        ),
        row=2, col=1
    )
    
    # Max Drawdown
    fig_sensitivity.add_trace(
        go.Scatter(
            x=method_results['lookback'],
            y=method_results['max_drawdown'] * 100,
            mode='lines+markers',
            name='Max Drawdown (%)',
            line=dict(color='red', width=2),
            marker=dict(size=8),
            showlegend=False
        ),
        row=2, col=2
    )
    
    fig_sensitivity.add_trace(
        go.Scatter(
            x=[best_row['lookback']],
            y=[best_row['max_drawdown'] * 100],
            mode='markers',
            marker=dict(size=15, color='red', symbol='star'),
            showlegend=False
        ),
        row=2, col=2
    )
    
    # Update layout
    fig_sensitivity.update_layout(
        height=600,
        title=f"Analisi Sensibilità Parametri - Metodo {best_method.upper()}",
        showlegend=True
    )
    
    # Update x-axis labels
    for i in range(1, 3):
        for j in range(1, 3):
            fig_sensitivity.update_xaxes(title_text="Lookback Period (giorni)", row=i, col=j)
    
    fig_sensitivity.show()
    
    # Identifica range di lookback stabili
    top_30_percent = method_results.nlargest(int(len(method_results) * 0.3), 'sharpe_ratio')
    stable_range = [top_30_percent['lookback'].min(), top_30_percent['lookback'].max()]
    
    print(f"\n📈 RANGE PARAMETRI STABILI:")
    print(f"   I migliori 30% delle configurazioni usano lookback tra {stable_range[0]} e {stable_range[1]} giorni")
    print(f"   Lookback consigliati: {sorted(top_30_percent['lookback'].tolist())}")


ANALISI SENSIBILITÀ PARAMETRI - METODO: SHARPE_MOMENTUM

Performance per diversi lookback (metodo sharpe_momentum):
----------------------------------------------------------------------
Lookback   5: Sharpe=-0.416 | Annual= -0.16% | Vol= 0.38% | MaxDD= -2.99%
Lookback  10: Sharpe=-0.407 | Annual= -0.16% | Vol= 0.39% | MaxDD= -3.03%
Lookback  15: Sharpe=-0.185 | Annual= -0.07% | Vol= 0.38% | MaxDD= -2.01%
Lookback  20: Sharpe= 0.098 | Annual=  0.03% | Vol= 0.35% | MaxDD= -1.60%
Lookback  30: Sharpe= 0.315 | Annual=  0.11% | Vol= 0.36% | MaxDD= -1.57%
Lookback  45: Sharpe= 0.248 | Annual=  0.09% | Vol= 0.36% | MaxDD= -1.81%
Lookback  60: Sharpe= 0.296 | Annual=  0.10% | Vol= 0.35% | MaxDD= -1.30%
Lookback  90: Sharpe= 0.260 | Annual=  0.09% | Vol= 0.34% | MaxDD= -1.23%
Lookback 120: Sharpe= 0.306 | Annual=  0.10% | Vol= 0.33% | MaxDD= -0.66%
Lookback 180: Sharpe= 0.641 | Annual=  0.21% | Vol= 0.32% | MaxDD= -0.63% ⭐ BEST

📊 STATISTICHE DI SENSIBILITÀ:
   Sharpe Ratio - Media: 0.116, St


📈 RANGE PARAMETRI STABILI:
   I migliori 30% delle configurazioni usano lookback tra 30 e 180 giorni
   Lookback consigliati: [30, 120, 180]


In [20]:
# Test rapido della nuova strategia Sharpe-Adjusted Momentum
print("="*80)
print("🚀 TEST RAPIDO: CONFRONTO MOMENTUM vs SHARPE-ADJUSTED MOMENTUM")
print("="*80)

# Test con parametri fissi per confronto diretto
test_lookback = 30
test_methods = ['momentum', 'sharpe_momentum']

comparison_results = []

for method in test_methods:
    print(f"\n🔍 Testing {method.upper()}...")
    result = rebalancer.backtest_strategy(test_lookback, method)
    
    # Calcola metriche
    portfolio_data = result['portfolio_data']
    total_ret, annual_ret, vol, sharpe, max_dd = calculate_performance_metrics(portfolio_data['returns'])
    
    comparison_results.append({
        'Method': method.replace('_', ' ').title(),
        'Total Return': f"{total_ret:.2%}",
        'Annual Return': f"{annual_ret:.2%}",
        'Volatility': f"{vol:.2%}",
        'Sharpe Ratio': f"{sharpe:.3f}",
        'Max Drawdown': f"{max_dd:.2%}",
        'Final Value': f"{result['final_value']:.2f}"
    })

# Crea tabella di confronto
comparison_df = pd.DataFrame(comparison_results)
print(f"\n📊 CONFRONTO DIRETTO (Lookback = {test_lookback} giorni):")
print("-" * 90)
print(comparison_df.to_string(index=False))

# Mostra i pesi finali per entrambe le strategie
print(f"\n💼 ALLOCAZIONI FINALI:")
print("-" * 50)

for method in test_methods:
    result = rebalancer.backtest_strategy(test_lookback, method)
    weights_df = result['weights']
    last_weights = weights_df.iloc[-1].sort_values(ascending=False)
    
    print(f"\n{method.replace('_', ' ').title()}:")
    for asset, weight in last_weights.items():
        if weight > 0.001:  # Mostra solo pesi significativi
            print(f"  {asset}: {weight:.1%}")
    
    # Conta quante strategie sono attive
    active_strategies = sum(1 for w in last_weights if w > 0.001)
    print(f"  → Strategie attive: {active_strategies}/{len(last_weights)}")

print(f"\n✨ La strategia Sharpe-Adjusted dovrebbe mostrare un miglior rapporto rischio/rendimento!")

🚀 TEST RAPIDO: CONFRONTO MOMENTUM vs SHARPE-ADJUSTED MOMENTUM

🔍 Testing MOMENTUM...

🔍 Testing SHARPE_MOMENTUM...

📊 CONFRONTO DIRETTO (Lookback = 30 giorni):
------------------------------------------------------------------------------------------
         Method Total Return Annual Return Volatility Sharpe Ratio Max Drawdown Final Value
       Momentum        0.60%         0.06%      0.40%        0.159       -1.74%        1.01
Sharpe Momentum        1.08%         0.11%      0.36%        0.315       -1.57%        1.01

💼 ALLOCAZIONI FINALI:
--------------------------------------------------

Momentum:
  AUDJPY: 34.8%
  AUDNZD: 21.2%
  NZDJPY: 20.1%
  EURCHF: 12.7%
  EURUSD: 10.4%
  USDCHF: 0.8%
  → Strategie attive: 6/10

Sharpe Momentum:
  AUDNZD: 36.7%
  AUDJPY: 24.5%
  NZDJPY: 16.1%
  EURCHF: 16.0%
  EURUSD: 6.2%
  USDCHF: 0.6%
  → Strategie attive: 6/10

✨ La strategia Sharpe-Adjusted dovrebbe mostrare un miglior rapporto rischio/rendimento!
